<a href="https://colab.research.google.com/github/oscar-horizonsarchitecture/llm-examples/blob/main/colab_files/HA_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HA Agents

In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install gradio
!pip install wikipedia


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 68.0 MB/s eta 0:00:00
    

In [2]:
import openai
from google.colab import userdata
from langchain.tools import tool

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory



openai.api_key = userdata.get('OPENAI_KEY')

In [12]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [16]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param


tools = [search_wikipedia]
#tools = []


legacy_prompt = ''' You are an intelligent agent to help people create their Legacy based on HA
### HA Legacy Maker
The main idea of this task is to help a user define a Legacy. Many users find articulating their Legacies a intimidating task.

This is the definition of LEGACY in HA:
- LEGACY: Interpreted definition Legacy: This dimension anchors the entire framework. It's the 'why' behind every endeavor. Ensuring that an endeavor leaves a lasting positive impact aligns with many contemporary theories on long-term value creation and sustainability. In a world where short-term gains are often prioritized, focusing on a Legacy ensures strategic foresight and long-term thinking. Official definition of Legacy: The description of the desired outcome, objective, or goal that the user(s) of the endeavor strives to achieve in a complex endeavor. This Legacy may be economic, social, environmental, personal, or collective, or a combination of these configurations. The concept of creating value that can be transferred refers to the idea that the outcome of the complex endeavor should have a lasting impact that will continue to benefit in the future. This transfer could take the form of financial assets, knowledge, resources, systems, or institutions that will have ongoing positive impacts or knowledge or skills that can be passed down to future generations. Legacy should be something that is created.

TASK 01: Make it conversational and adaptive. Generate a list of questions, one at a time. Make one question, wait for the answer and based on this information, generate the next question. These questions should be done in a friendly and empathic manner by a HA user. Generate questions so that you, GPT4 gather all the necessary information to generate a strong, clear and inspirational Legacy, to then send it to the Legacy Enhancer. Decide the number of questions until you have covered all the elements in both definitions:

Finish asking the user if he/she agrees on the generated Legacy and if he/she has feedback to make another iteration to improve it based on the user desires. If yes, then ask if the length and tone are the right ones?

If yes, then tell he/she that now we can move on to the next dimensions. If not, adjust.

In general, assist the user to generate their Legacy until it is satisfied in a conversational manner.
'''

class cbfs(param.Parameterized):

    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0.5,openai_api_key=userdata.get('OPENAI_KEY'), model="gpt-3.5-turbo").bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", legacy_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)


    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=650)),
            pn.Row('AI Agent:', pn.pane.Markdown(self.answer, width=650, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)

    def invoque_agent(self, message):
      if not message:
            return
      inp.value = ''
      result = self.qa.invoke({"input": message})
      self.answer = result['output']
      return result['output']


    def clr_history(self,count=0):
        self.chat_history = []
        return


In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# AI Agent - HA')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

## Gradio

In [ ]:
import gradio as gr

cb2 = cbfs(tools)

def respond_to_input(message, history):
    response = cb2.invoque_agent(message)
    return response

#r= respond_to_input("Hola", "")
#print(r)

chat_interface = gr.ChatInterface(respond_to_input).queue()
chat_interface.launch(share=True, debug=True)  #Está presentando errores, parece que sólo funciona en share true y debug false

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://03722658dee1b3330e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  "error": {
    "message": "The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID e0b075f88341f9748d7dcd157df5323f in your email.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID e0b075f88341f9748d7dcd157df5323f in your email.)', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Fri, 03 Nov 2023 20:37:43 GMT', 'Content-Type': 'application/json', 'Content-Length': '366', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-organization': 'user-rvon08sxmfkux9k9bhiunf1a', 'openai-processing-ms': '368', 'openai-version': '2020-10-01', 'st